# OpenStreetMap Data Case Study

### by Murat Guercue

## Map Area

Orlando / Florida, USA.

•http://www.openstreetmap.org/relation/1128379#map=12/28.4815/-81.3675

This map is of my favorite city Orlando (USA/Florida), so I'd like to explore open-source map of this area, reveal some inconsistencies in data and contribute to its improvement on OpenStreetMap.org.

As described in the introductions, my first action is to split the Orldons.osm File into a sample file.


In [15]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "Openstreetmap_Orlando.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 30 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/
    inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

    # changed code for Python 3
with open(SAMPLE_FILE, 'wb') as output:
    output.write(bytes('<?xml version="1.0" encoding="UTF-8"?>\n', 'UTF-8'))
    output.write(bytes('<osm>\n  ', 'UTF-8'))

    # Write every 10th top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % 10 == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(bytes('</osm>', 'UTF-8'))

# Problems entcountered

## Data Auditing

Our first step is to get an overview of our dataset, to be able to audit the Orlando_osm file. Therefore, we count the tag content of the dataset.


In [16]:
import xml.etree.cElementTree as ET 
import pprint
 
OSM_FILE = "Openstreetmap_Orlando.osm" 
 
def count_tags(filename): 
    tags = {} 
    for event, elem in ET.iterparse(filename): 
        if elem.tag in tags:  
            tags[elem.tag] += 1
        else: 
            tags[elem.tag] = 1
    return tags 
            
pprint.pprint(count_tags(OSM_FILE))

{'member': 23954,
 'meta': 1,
 'nd': 331444,
 'node': 270522,
 'note': 1,
 'osm': 1,
 'relation': 663,
 'tag': 119477,
 'way': 26206}


As we can see our Orlando_osm dataset has following tags:

    - member: 23954
    - meta: 1
    - nd: 331444
    - node: 270522
    - note: 1
    - osm: 1
    - relation: 663
    - tag: 119477
    - way: 26206

The content of our "k" values are different. Now we will check what is the structure of "k" value and which are valid or not.

In [17]:
import xml.etree.cElementTree as ET 
import pprint 
import re 
 
from collections import defaultdict 
 
lower = re.compile(r'^([a-z]|_)*$') 
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$') 
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]') 

 
OSM_FILE = "Openstreetmap_Orlando.osm" 

 
def key_type(element, keys): 
    if element.tag == "tag": 
        for tag in element.iter('tag'): 
            k = tag.get('k') 
            if lower.search(element.attrib['k']): 
                keys['lower'] = keys['lower'] + 1 
            elif lower_colon.search(element.attrib['k']): 
                keys['lower_colon'] = keys['lower_colon'] + 1 
            elif problemchars.search(element.attrib['k']): 
                keys['problemchars'] = keys['problemchars'] + 1 
            else: 
                keys['other'] = keys['other'] + 1 
      
    return keys 

def process_map(filename): 
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0} 
    for _, element in ET.iterparse(filename): 
        keys = key_type(element, keys) 
 
    return keys 

pprint.pprint(process_map(OSM_FILE)) 


{'lower': 81188, 'lower_colon': 33527, 'other': 4762, 'problemchars': 0}


We can identify the follwing structure at our Orlando_osm dataset.


• "lower" : 81188 , for valid tags with only lowercase letters,

• "lower_colon" : 33527 , for tags with a colon which are also valid,

• "problemchars" : 0 , for tags with special/problematic characters, and

• "other" : 4762 , for other which are outside of the other groups.



Now we will focus on problems of our dataset. If we check the Street Names we can assess different structures. We can see some different abbreviations and different writing structures. No we will harmonize them. 

In [18]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSM_FILE = "Openstreetmap_Orlando.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Highway"]

# Abbreviations
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Hwy": "Highway",
            "Blvd": "Boulevard",
            "Rd.": "Road"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    m = street_type_re.search(name)
    if m not in expected:
        name = re.sub(m.group(), mapping[m.group()], name)
    
    num_line_street_re = re.compile(r'\d0?(st|nd|rd|th|)\s(Line)$', re.IGNORECASE)
    num_line_mapping = {
                         "13th": "Thirteen",
                         "19th": "Nineteen"
                       }
    
    if num_line_street_re.match(name):
        nth = nth_re.search(name)
        name = num_line_mapping[nth.group(0)] + " Line"

    return name


def test():
    st_types = audit(OSM_FILE)
    assert len(st_types) == 3
    pprint.pprint(dict(st_types))
    test()

After converting the Street Names we will now focus on phone numbers and harmonize them to the structure +1 XXX XXX XXXX. Our origin dataset different structures with brackets, with minus or without any space.

In [19]:
def update_phone_num(phone_num):
    # Check for valid phone number format
    p = PHONENUM.match(phone_num)
    if p is None:
        # Remove all brackets
        if "(" in phone_num or ")" in phone_num:
            phone_num = re.sub("[()]", "", phone_num)
        # Convert all dashes to spaces
        if "-" in phone_num:
            phone_num = re.sub("-", " ", phone_num)
        # Space out 10 straight numbers
        if re.match(r'\d{10}', phone_num) is not None:
            phone_num = phone_num[:3] + " " + phone_num[3:6] + " " + phone_num[6:]
        # Space out 11 straight numbers
        elif re.match(r'\d{11}', phone_num) is not None:
            phone_num = phone_num[:1] + " " + phone_num[1:4] + " " + phone_num[4:7] \
            			+ " " + phone_num[7:]
        # Add full country code
        if re.match(r'\d{3}\s\d{3}\s\d{4}', phone_num) is not None:
            phone_num = "+1 " + phone_num
        # Add + in country code
        elif re.match(r'1\s\d{3}\s\d{3}\s\d{4}', phone_num) is not None:
            phone_num = "+" + phone_num
        # Ignore tag if no area code and local number (<10 digits)
        elif sum(c.isdigit() for c in phone_num) < 10:
            return None
    return phone_num

# Data Overview

Now we will generate some general information about our dataset.

First step is to prepare the dataset to be inserted into a SQL database.
To do so we will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

In [20]:
import csv 
import codecs 
import re 
import xml.etree.cElementTree as ET 
from unittest import TestCase 
 
import cerberus 
import jsonschema 
 
OSM_PATH = "Openstreetmap_Orlando.osm" 
 
NODES_PATH = "nodes.csv" 
NODE_TAGS_PATH = "nodes_tags.csv" 
WAYS_PATH = "ways.csv" 
WAY_NODES_PATH = "ways_nodes.csv" 
WAY_TAGS_PATH = "ways_tags.csv" 
 
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+') 
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]') 
 
SCHEMA = jsonschema.jsonschema 
 
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp'] 
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type'] 
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp'] 
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type'] 
WAY_NODES_FIELDS = ['id', 'node_id', 'position'] 
 
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS, 
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'): 
    """Clean and shape node or way XML element to Python dict""" 
 
    node_attribs = {} 
    way_attribs = {} 
    way_nodes = [] 
    tags = []   

    if element.tag == 'node': 
        for attrib in element.attrib: 
            if attrib in NODE_FIELDS: 
                node_attribs[attrib] = element.attrib[attrib] 
       
        for child in element: 
            node_tag = {} 
            if LOWER_COLON.match(child.attrib['k']): 
                node_tag['type'] = child.attrib['k'].split(':',1)[0] 
                node_tag['key'] = child.attrib['k'].split(':',1)[1] 
                node_tag['id'] = element.attrib['id'] 
                node_tag['value'] = child.attrib['v'] 
                tags.append(node_tag) 
            elif PROBLEMCHARS.match(child.attrib['k']): 
                continue 
            else: 
                node_tag['type'] = 'regular' 
                node_tag['key'] = child.attrib['k'] 
                node_tag['id'] = element.attrib['id'] 
                node_tag['value'] = child.attrib['v'] 
                tags.append(node_tag) 
        
        return {'node': node_attribs, 'node_tags': tags} 
         
    elif element.tag == 'way': 
        for attrib in element.attrib: 
            if attrib in WAY_FIELDS: 
                way_attribs[attrib] = element.attrib[attrib] 
        
        position = 0 
        for child in element: 
            way_tag = {} 
            way_node = {} 
            
            if child.tag == 'tag': 
                if LOWER_COLON.match(child.attrib['k']): 
                    way_tag['type'] = child.attrib['k'].split(':',1)[0] 
                    way_tag['key'] = child.attrib['k'].split(':',1)[1] 
                    way_tag['id'] = element.attrib['id'] 
                    way_tag['value'] = child.attrib['v'] 
                    tags.append(way_tag) 
                elif PROBLEMCHARS.match(child.attrib['k']): 
                    continue 
                else: 
                    way_tag['type'] = 'regular' 
                    way_tag['key'] = child.attrib['k'] 
                    way_tag['id'] = element.attrib['id'] 
                    way_tag['value'] = child.attrib['v'] 
                    tags.append(way_tag) 
                     
            elif child.tag == 'nd': 
                way_node['id'] = element.attrib['id'] 
                way_node['node_id'] = child.attrib['ref'] 
                way_node['position'] = position 
                position += 1 
                way_nodes.append(way_node) 
       
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags} 

# ================================================== # 
#               Helper Functions                     # 
# ================================================== # 
def get_element(osm_file, tags=('node', 'way', 'relation')): 
    """Yield element if it is the right type of tag""" 

    context = ET.iterparse(osm_file, events=('start', 'end')) 
    _, root = next(context) 
    for event, elem in context: 
        if event == 'end' and elem.tag in tags: 
            yield elem 
            root.clear() 


def validate_element(element, validator, schema=SCHEMA): 
    """Raise ValidationError if element does not match schema""" 
    if validator.validate(element, schema) is not True: 
        field, errors = next(validator.errors.iteritems()) 
        message_string = "\nElement of type '{0}' has the following errors:\n{1}" 
        error_strings = ( 
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v)) 
            for k, v in errors.iteritems() 
        ) 
        raise cerberus.ValidationError( 
            message_string.format(field, "\n".join(error_strings)) 
        ) 

class UnicodeDictWriter(csv.DictWriter, object): 
    """Extend csv.DictWriter to handle Unicode input""" 

    def writerow(self, row): 
        super(UnicodeDictWriter, self).writerow({ 
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems() 
        }) 

    def writerows(self, rows): 
        for row in rows: 
            self.writerow(row) 
 
 
# ================================================== # 
#               Main Function                        # 
# ================================================== # 
def process_map(file_in, validate): 
    """Iteratively process each XML element and write to csv(s)""" 
    with codecs.open(NODES_PATH, 'w') as nodes_file,\
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file,\
         codecs.open(WAYS_PATH, 'w') as ways_file,\
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file,\
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file: 
 
 
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS) 
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS) 
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS) 
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS) 
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS) 

        nodes_writer.writeheader() 
        node_tags_writer.writeheader() 
        ways_writer.writeheader() 
        way_nodes_writer.writeheader() 
        way_tags_writer.writeheader() 
 
        validator = cerberus.Validator() 
 
        for element in get_element(file_in, tags=('node', 'way')): 
            el = shape_element(element) 
            if el: 
                if validate is True: 
                    validate_element(el, validator) 

 
                if element.tag == 'node': 
                    nodes_writer.writerow(el['node']) 
                    node_tags_writer.writerows(el['node_tags']) 
                elif element.tag == 'way': 
                    ways_writer.writerow(el['way']) 
                    way_nodes_writer.writerows(el['way_nodes']) 
                    way_tags_writer.writerows(el['way_tags']) 

if __name__ == '__main__': 
    process_map(OSM_PATH, validate=True) 

ImportError: No module named 'cerberus'

After perparing our dataset and creating the csv files we will investigate now our dataset and provide some general statistics by using SQL queries.

Firt we start with file sizes:

Now let´s focus on number of Ways and Nodes.

Ways:

<div class="alert alert-block alert-info"> _sqlite> SELECT COUNT(*) FROM Ways;_ </div>

XXXX

Nodes:

<div class="alert alert-block alert-info"> _sqlite> SELECT COUNT(*) FROM Nodes;_ </div>

XXXX

Top 20 Contributers:

<div class="alert alert-block alert-info"> sqlite> SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 20; </div> 

# Additional Data Exploration

Top 5 Cuisines

<div class="alert alert-block alert-info"> sqlite> SELECT nodesTags.value, COUNT(*) as num

FROM nodesTags

JOIN (SELECT DISTINCT(id) FROM nodesTags WHERE value='restaurant') i

ON nodesTags.id=i.id

WHERE nodesTags.key='cuisine'    

GROUP BY nodesTags.value

ORDER BY num DESC

LIMIT 5; </div>

XXXX

Biggest Religion:

<div class="alert alert-block alert-info"> sqlite> SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 1; </div>

# Conclusion

Our Orlando dataset is despite of huge size in general of good quality but showing some typical differences/errors which needs harmonization and adjustments. By huge number of Users contributing to the OpenStreetMaps it´s very likely to figure out human made typo errors. For typo errors like street name abbreviations and different phone number structures we used codes learned at our lessons to harmonize and adjust them.

After parsing our dataset we could also find some interesting statistics of Orlando.

In general our dataset is very useful, but needs some adjustments in some areas.